### Second: provide SQL queries
Answer three of the following questions with at least one question coming from the closed-ended and one from the open-ended question set. Each question should be answered using one query.
Closed-ended questions:
* What are the top 5 brands by receipts scanned among users 21 and over?
    1. 'NERDS CANDY'
    2. 'DOVE'
    3. 'TRIDENT'
    4. 'SOUR PATCH KIDS'
    5. 'MEIJER'
* What are the top 5 brands by sales among users that have had their account for at least six months?
    1. 'CVS'
    2. 'DOVE'
    3. 'TRIDENT'
    4. 'COORS LIGHT'
    5. 'TRESEMMÉ'
3. What is the percentage of sales in the Health & Wellness category by generation?
    * 'Zoomers', 0.0
    * 'Millenials', 19.43
    * 'X',16.88
    * 'Boomers', 27.66
    * 'Silent', 0.0
    * 'Other', 24.16 - missing birth_date

Open-ended questions: for these, make assumptions and clearly state them when answering the question.
1. Who are Fetch’s power users?
2. Which is the leading brand in the Dips & Salsa category? - Tostitos*
    * Long answer: The dataset here is incomplete, as the transactions are missing a substantial amount of barcodes. Based on the barcodes present, Tostitos is the leading brand by 3 metrics - number of sales, sum of sales in $, and number of receipts. This also lines up with public sales records, such as an older report from Statista, as well as personal biases when browsing this category in stores. https://www.statista.com/statistics/300313/us-leading-salsa-based-on-dollar-sales/#:~:text=Tostitos%20is%20the%20leading%20brand,owned%20and%20operated%20by%20PepsiCo.
3. At what percent has Fetch grown year over year? - not enough data, only 4 months of 1 year is available

### Importing packages and data into environment

In [2]:
import sqlite3
import pandas as pd

# Connect to SQLite (create the database file if it doesn't exist)
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# List of CSV file paths and corresponding table names
csv_files = [
    ('USER_TAKEHOME.csv', 'users'),
    ('PRODUCTS_TAKEHOME.csv', 'products'),
    ('transactions_updated.csv', 'transactions')
]

# Function to create a table and import CSV data
def import_csv_to_sql(csv_file, table_name):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)
    
    # Create the SQL table if it doesn't exist, using the DataFrame columns as SQL columns
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Data from {csv_file} imported into {table_name}")

# Loop through the list of CSV files and import each one
for csv_file, table_name in csv_files:
    import_csv_to_sql(csv_file, table_name)

# Commit and close the connection
conn.commit()
conn.close()

Data from USER_TAKEHOME.csv imported into users
Data from PRODUCTS_TAKEHOME.csv imported into products
Data from transactions_updated.csv imported into transactions


### What are the top 5 brands by receipts scanned among users 21 and over?
1. 'NERDS CANDY'
2. 'DOVE'
3. 'TRIDENT'
4. 'SOUR PATCH KIDS'
5. 'MEIJER'

In [4]:
# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select p.brand, 
       count(distinct(t.receipt_id)) as receipts
from transactions t
left join products p
    on t.barcode = p.barcode
where user_id in (
    SELECT ID
    FROM users 
    WHERE BIRTH_DATE < DATE('now','-21 years') 
)
and brand <> 'None'  -- result depends on whether you want to include/exclude 'None'
group by p.brand
order by receipts desc
limit 5
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('NERDS CANDY', 3)
('DOVE', 3)
('TRIDENT', 2)
('SOUR PATCH KIDS', 2)
('MEIJER', 2)


### 2. What are the top 5 brands by sales among users that have had their account for at least six months?
    1. 'CVS'
    2. 'DOVE'
    3. 'TRIDENT'
    4. 'COORS LIGHT'
    5. 'TRESEMMÉ'

In [6]:
# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select p.brand, count(distinct(t.receipt_id)), sum(final_sale)
from transactions t
left join products p
    on t.barcode = p.barcode
where user_id in (
    SELECT ID
    FROM users 
    WHERE CREATED_DATE < DATE('now','-6 months') 
)
and brand <> 'None' --result depends on if you want to include / exclude 'None'
group by 1
order by 3 desc
LIMIT 5
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('CVS', 1, 144.0)
('DOVE', 3, 61.82)
('TRIDENT', 2, 46.72)
('COORS LIGHT', 1, 34.96)
('TRESEMMÉ', 1, 29.16)


### 3. What is the percentage of sales in the Health & Wellness category by [age] generation?
* Silent, 0.0
* Boomers, 37.31
* X, 22.36
* Millenials, 19.24
* Zoomers, 0
* Other, 24.9 - missing birth_date

In [8]:
# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select case when birth_date >= '2022-01-01' then '8.Alpha'
    when birth_date >= '1997-01-01' then '7.Zoomers'
    when birth_date >= '1981-01-01' then '6.Millenials'
    when birth_date >= '1965-01-01' then '5.X'
    when birth_date >= '1946-01-01' then '4.Boomers'
    when birth_date >= '1928-01-01' then '3.Silent'
    when birth_date >= '1901-01-01' then '2.Greatest'
    when birth_date >= '1883-01-01' then '1.Lost'
    else 'Other'
    end as generation,
--sum(case when category_1 = 'Health & Wellness' then final_sale else 0 end), 
--sum(final_sale),
round(100.0*sum(case when category_1 = 'Health & Wellness' then final_sale else 0 end) / sum(final_sale),2) as health_perc
from transactions t
left join products p
    on t.barcode = p.barcode
left join users u
    on u.id = t.user_id
group by 1
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('3.Silent', 0.0)
('4.Boomers', 37.31)
('5.X', 22.36)
('6.Millenials', 19.24)
('7.Zoomers', 0.0)
('Other', 24.9)


In [9]:
# Confirming that 'Other' lines up with missing birth_dates

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select 
    sum(case when category_1 = 'Health & Wellness' then final_sale else 0 end), 
    sum(final_sale),
    round(100.0*sum(case when category_1 = 'Health & Wellness' then final_sale else 0 end) / sum(final_sale),4) as health_perc
from transactions t
left join products p
    on t.barcode = p.barcode
left join users u
    on u.id = t.user_id
where birth_date is null
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

(54777.04, 219966.32, 24.9025)


### 2.2. Which is the leading brand in the Dips & Salsa category?
Tostitos
* Long answer: The dataset here is incomplete, as the transactions are missing a substantial amount of barcodes. Based on the barcodes present, Tostitos is the leading brand by 3 metrics - number of sales, sum of sales in $, and number of receipts. This also lines up with public sales records, such as an older report from Statista, as well as personal biases when browsing this category in stores. https://www.statista.com/statistics/300313/us-leading-salsa-based-on-dollar-sales/#:~:text=Tostitos%20is%20the%20leading%20brand,owned%20and%20operated%20by%20PepsiCo.

In [11]:
# View leading brands by number of sales from customer receipts in our data

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select 
brand, count(distinct(receipt_id)), sum(final_quantity), round(sum(final_sale),2)
from transactions t
left join products p
    on t.barcode = p.barcode
where category_2 = 'Dips & Salsa'
group by 1
order by 4 desc
limit 10
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('TOSTITOS', 36, 76.0, 362.32)
(None, 21, 44.0, 201.94)
('GOOD FOODS', 9, 18.0, 189.82)
('PACE', 24, 48.0, 171.15)
('FRITOS', 19, 40.0, 134.72)
('MARKETSIDE', 16, 32.0, 130.73)
('HELUVA GOOD!', 15, 30.0, 106.56)
('MARZETTI', 11, 22.0, 102.47)
('FRESHNESS GUARANTEED', 12, 24.0, 93.32)
("DEAN'S DAIRY DIP", 17, 34.0, 79.9)


In [12]:
# View number of products in dataset

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select brand, count(1)
from products p
where category_2 = 'Dips & Salsa'
group by 1
order by 2 desc
limit 10
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

(None, 6329)
('PRIVATE LABEL', 1166)
('SABRA', 268)
('WHOLLY', 217)
('TOSTITOS', 149)
('PACE', 119)
('GOOD FOODS', 115)
('MEIJER', 112)
('MARZETTI', 109)
('BRAND NOT KNOWN', 108)


### Miscellaneous tests

In [14]:
# Missing user data - check to confirm that most user_ids are missing from user table when joining

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select 
count(distinct(case when u.id is null then user_id end)) as missing_userid,
count(distinct(user_id)) as total_users
from transactions t
left join users u
    on u.id = t.user_id
limit 100
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

(17603, 17694)


In [15]:
# Missing user data - check to confirm that most user_ids are missing from user table when joining

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select 
min(birth_date), max(birth_date)
from transactions t
left join users u
    on t.user_id = u.id
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('1943-09-03 05:00:00.000 Z', '1997-02-25 00:00:00.000 Z')


### Exploration
These queries are irrelevant to the questions asked, but are for personal exploration, and identifying trends in the data.

In [17]:
# Check most popular stores

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select store_name, count(distinct(receipt_id)), sum(final_quantity), sum(final_sale),
round(100.0*round(sum(final_sale),2) / (select sum(final_sale) from transactions),2) as perc_of_sales
from transactions
group by 1
order by 4 desc
limit 10
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

# Check most popular stores

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select category_1, count(1), sum(final_quantity), round(sum(final_sale),2),
round(100.0*round(sum(final_sale),2) / (select sum(final_sale) from transactions),2) as perc_of_sales
from products p
left join transactions t
    on p.barcode = t.barcode
group by 1
order by 4 desc
limit 10
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

('WALMART', 10347, 22253.1, 82505.55, 37.31)
('COSTCO', 510, 996.0, 13200.36, 5.97)
("SAM'S CLUB", 504, 1015.0, 11140.130000000001, 5.04)
('TARGET', 739, 1640.0, 9239.36, 4.18)
('DOLLAR GENERAL STORE', 1360, 2981.0, 8918.210000000001, 4.03)
('WALGREENS', 559, 1242.0, 8365.08, 3.78)
('CVS', 579, 1056.0, 7921.91, 3.58)
('ALDI', 1278, 2337.77, 5701.68, 2.58)
('MEIJER', 576, 1471.62, 5449.69, 2.46)
('KROGER', 725, 1451.38, 5448.5, 2.46)
('Snacks', 336271, 16001.32, 61706.24, 27.91)
('Health & Wellness', 516969, 7114.6, 55096.12, 24.92)
('Beverages', 5465, 1911.7, 8707.5, 3.94)
(None, 808, 758.47, 3181.29, 1.44)
('Alcohol', 577, 113.0, 1620.99, 0.73)
('Pantry', 1010, 209.34, 829.67, 0.38)
('Dairy', 722, 169.0, 571.03, 0.26)
('Deli & Bakery', 86, 28.0, 149.2, 0.07)
('Frozen', 84, 28.0, 139.09, 0.06)
('Needs Review', 561, 22.0, 120.66, 0.05)


In [18]:
# Check most popular stores

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
select category_3, count(1), sum(final_quantity), round(sum(final_sale),2)
from products p
left join transactions t
    on p.barcode = t.barcode
group by 1
order by 4 desc
limit 10
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

(None, 62678, 2803.85, 14120.74)
('Cola', 3859, 1685.7, 8124.44)
('Chocolate Candy', 49403, 2341.99, 7070.02)
('Vitamins & Herbal Supplements', 55930, 429.6, 5852.23)
('Potato Chips', 12258, 1327.59, 4460.72)
('Confection Candy', 57934, 1430.37, 3775.49)
('Tortilla Chips', 4624, 840.18, 3282.59)
('Allergy & Sinus Medicines & Treatments', 5922, 220.0, 2960.52)
('Body Wash', 15306, 367.0, 2799.76)
('Digestive Health Medicines & Treatments', 9755, 276.0, 2497.02)


In [19]:
# Missing user data - check to confirm that most user_ids are missing from user table when joining
# transactions contains 17,694 ids, users contains 100,000, only 91 show up in both

# Reconnect to the database
conn = sqlite3.connect('local_db.db')
cursor = conn.cursor()

# Query the data from a table
query = """
SELECT
    -- Count distinct user_id not in users table
    (SELECT COUNT(DISTINCT t.user_id)
     FROM transactions t
     LEFT JOIN users u ON t.user_id = u.id
     WHERE u.id IS NULL) AS t_notin_u,

    -- Count distinct id not in transactions table
    (SELECT COUNT(DISTINCT u.id)
     FROM users u
     LEFT JOIN transactions t ON u.id = t.user_id
     WHERE t.user_id IS NULL) AS u_notin_t,

    -- Count distinct user_id in transactions table
    (SELECT COUNT(DISTINCT user_id)
     FROM transactions) AS trxn_ids,

    -- Count distinct id in users table
    (SELECT COUNT(DISTINCT id)
     FROM users) AS u_ids
"""
cursor.execute(query)

# Fetch and print the result
result = cursor.fetchall()
for row in result:
    print(row)

# Close the connection
conn.close()

(17603, 99909, 17694, 100000)
